In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import pytorch_lightning as pl

import sys
sys.path.insert(0,'..')

from starreco.model import *
#from starreco.data import *

%load_ext autoreload
%autoreload 2

In [2]:
user_x = torch.FloatTensor(3760,15).random_(0, 5)
user_train_ds = TensorDataset(user_x, user_x)
user_train_dl = DataLoader(user_train_ds, batch_size = 1024, num_workers = 8)

user_dae = DAE(15, [10, 5])

pl.Trainer(gpus = 1, max_epochs = 100, progress_bar_refresh_rate = 50, logger = False)\
.fit(user_dae, user_train_dl)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
/home/kyleong/anaconda3/envs/starreco/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name    | Type                  | Params
--------------------------------------------------
0 | encoder | MultilayerPerceptrons | 245   
1 | dropout | Dropout               | 0     
2 | decoder | MultilayerPerceptrons | 245   
--------------------------------------------------
490       Trainable params
0         Non-trainable params
490       Total params
0.002     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 4/4 [00:00<00:00, 19.56it/s, loss=1.86, train_loss_step=1.840, train_loss_epoch=1.860]


1

In [3]:
item_x = torch.FloatTensor(6040, 15).random_(0, 5)
item_train_ds = TensorDataset(item_x, item_x)
item_train_dl = DataLoader(item_train_ds, batch_size = 1024, num_workers = 8)

item_ae = AE(15, 5)

pl.Trainer(gpus = 1, max_epochs = 100, progress_bar_refresh_rate = 50, logger = False)\
.fit(item_ae, item_train_dl)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name    | Type                  | Params
--------------------------------------------------
0 | encoder | MultilayerPerceptrons | 90    
1 | dropout | Dropout               | 0     
2 | decoder | MultilayerPerceptrons | 90    
--------------------------------------------------
180       Trainable params
0         Non-trainable params
180       Total params
0.001     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 6/6 [00:00<00:00, 24.74it/s, loss=2.41, train_loss_step=2.390, train_loss_epoch=2.400]


1

In [4]:
gmf_pp = GMFpp(user_dae, item_ae)
for name, param in user_dae.named_parameters():
    print(name, param)
print("---")
for name, param in item_ae.named_parameters():
    print(name, param)
print("---")
for name, param in gmf_pp.named_parameters():
    print(name, param)

encoder.mlp.0.weight Parameter containing:
tensor([[ 2.2360e-01,  1.1560e-01,  2.7189e-01,  6.3193e-02,  2.8762e-01,
         -2.5618e-02, -2.2983e-01,  5.1779e-04,  4.4871e-02,  3.2338e-02,
          1.9036e-01, -1.0512e-01,  9.3558e-02, -8.2610e-04,  2.1318e-02],
        [-2.9538e-01,  6.7786e-02, -4.9357e-03,  1.0450e-01, -1.9468e-01,
         -1.6195e-01,  3.1026e-01, -1.5295e-01,  7.2508e-02,  2.7020e-01,
         -5.7347e-02, -1.0240e-01, -3.1779e-02, -2.5899e-02, -2.4179e-02],
        [-2.9794e-02, -1.2798e-01, -6.2860e-03,  2.3442e-01, -4.1442e-02,
         -6.4121e-02, -9.9343e-04, -1.5924e-01, -2.4190e-01, -1.8241e-01,
         -1.0787e-01,  9.5596e-02, -1.7776e-01, -1.4052e-01,  9.8563e-03],
        [ 1.0427e-01,  8.6272e-02, -1.5128e-02, -1.5622e-01,  1.2761e-01,
          4.6706e-02, -2.2152e-01, -1.3803e-02,  1.6518e-01, -1.0977e-01,
          1.0845e-01, -3.7354e-01,  2.1796e-01,  1.2019e-02, -1.5457e-01],
        [ 3.3736e-03, -6.2444e-02, -1.7963e-02,  1.2707e-01, -1.6

In [5]:
X = torch.FloatTensor(1000000, 30).random_(0, 5)
y = torch.FloatTensor(1000000).random_(1,6)
train_ds = TensorDataset(X,y)
train_dl = DataLoader(train_ds, batch_size = 1024, num_workers = 8)

pl.Trainer(gpus = 1, max_epochs = 10, progress_bar_refresh_rate = 50, logger = False)\
.fit(gmf_pp, train_dl)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name    | Type                  | Params
--------------------------------------------------
0 | user_ae | DAE                   | 490   
1 | item_ae | AE                    | 180   
2 | mlp     | MultilayerPerceptrons | 6     
--------------------------------------------------
361       Trainable params
315       Non-trainable params
676       Total params
0.003     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 977/977 [00:09<00:00, 102.50it/s, loss=2.01, train_loss_step=1.980, train_loss_epoch=2.010]


1

In [6]:
for name, param in user_dae.named_parameters():
    print(name, param)
print("---")
for name, param in item_ae.named_parameters():
    print(name, param)
print("---")
for name, param in gmf_pp.named_parameters():
    print(name, param)

encoder.mlp.0.weight Parameter containing:
tensor([[ 0.1851, -0.0419,  0.3467,  0.5036,  0.3001, -0.0670, -0.1607, -0.0688,
         -0.1442, -0.0919,  0.1256, -0.2452, -0.1187, -0.1385,  0.1534],
        [-0.0428,  0.0825,  0.1784,  0.4331, -0.2240, -0.2091,  0.4281, -0.0825,
          0.0856,  0.2826, -0.2418,  0.0356, -0.0065, -0.0880,  0.0021],
        [ 0.0898, -0.3752,  0.2756,  0.9806,  1.0484, -0.2273,  0.3103, -0.1534,
          0.0625,  0.3066, -0.2741, -0.4275, -0.1675, -0.7780,  0.0284],
        [ 1.2469,  0.2008, -0.4176, -0.9350,  0.3628, -0.2386, -0.7396,  0.0302,
          0.3008, -0.3683,  0.6097,  0.4210,  0.2483, -0.1860, -0.1051],
        [ 0.4652, -0.4334,  0.0375,  1.4370,  1.2066, -0.3346, -0.3585, -0.7086,
         -0.3065,  0.1494, -0.5035,  0.1998, -0.4708, -0.1891,  0.0090],
        [ 0.0137,  0.0684, -0.0446,  0.2142,  0.2085,  0.1545,  0.5725, -0.1434,
          0.0357, -0.1518,  0.1857,  0.3258, -0.0814,  0.2500,  0.2755],
        [-0.4122,  0.3908, -0.009